### Install tendb
Must install latest tendb before running the commands
Install from released package from pypi
```bash
$ pip install -i https://test.pypi.org/simple/ tendb
```
Read local setup.py and install tendb
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb
```
Install from local wheel file
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb/dist/tendb-0.0.1-py3-none-any.whl
```

Read Arrow table

In [2]:
import pyarrow as pa
from pyarrow import csv

In [3]:
csv_options = pa.csv.ParseOptions(delimiter='|')
lineitem_table = pa.csv.read_csv(input_file='lineitem.tbl', parse_options=csv_options)
lineitem_table

pyarrow.Table
L_ORDERKEY: int64
L_PARTKEY: int64
L_SUPPKEY: int64
L_LINENUMBER: int64
L_QUANTITY: int64
L_EXTENDEDPRICE: double
L_DISCOUNT: double
L_TAX: double
L_RETURNFLAG: string
L_LINESTATUS: string
L_SHIPDATE: timestamp[s]
L_COMMITDATE: timestamp[s]
L_RECEIPTDATE: timestamp[s]
L_SHIPINSTRUCT: string
L_SHIPMODE: string
L_COMMENT: string

In [4]:
len(lineitem_table)

6005

In [5]:
isinstance(lineitem_table, object)

True

In [6]:
hex(id(lineitem_table))

'0x7fdb938d74b0'

In [7]:
lineitem_table.num_rows

6005

In [8]:
lineitem_table.num_columns

16

In [9]:
id(lineitem_table)

140581050086576

TenDB sample commands

In [10]:
import tendb as ten

In [11]:
tcache = ten.new_cache()

In [12]:
tcache

469773248

In [ ]:
ten.add_table(tcache, 'lineitem', id(lineitem_table))

In [ ]:
ten.query5()

In [ ]:
ten.test()

In [ ]:
f = ten.Foo()

In [ ]:
f.bar()